In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install "git+https://github.com/salaniz/pycocoevalcap.git"

  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-b3_1jvh4
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-b3_1jvh4
  Resolved https://github.com/salaniz/pycocoevalcap.git to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312246 sha256=ed69637085027f65b67ddd9568d511467db4402eb76309bd96b0262f7f055ff6
  Stored in directory: /tmp/pip-ephem-wheel-cache-_n55olc0/wheels/57/de/9f/7aa9ae75f4e4733cc4b66b5c3a65a05de62a28e8111649fee5
Successfully built pycocoevalcap


In [ ]:
import os
import json
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

# The path where MSVD test captions are located (from part4.1):
MSVD_test_caps_file = '/content/drive/My Drive/part3_and_4/MSVD_test_captions.json'
# The path where relevant MSVD generated test captions are located (from part3):
results_file = '/content/drive/My Drive/part3_and_4/MSVD16_1500/MSVD16_1500_test_generated_caps.json'
# The json file path where results of evaluation metrics are stored:
result_path = '/content/drive/My Drive/part3_and_4/MSVD16_1500/'

with open(MSVD_test_caps_file) as file:
    actual_caps = json.load(file) # actual caption dataset
with open(results_file) as file:
    res_caps = json.load(file) # generated captions by relevant models

dict_res = {}
dict_act = {}
for i in range(0, len(actual_caps)):
    dict_res[res_caps[i]['video_id']] = [res_caps[i]['caption']]
    cap_list = res_caps[i]['caption'].split()
    cap_length = len(cap_list)
    act_cap = ''
    for count in range(0, 15):
        for j in range(0,len(actual_caps[i]['captions'])):
            if len(actual_caps[i]['captions'][j][0].split()) <= (cap_length + count):
                act_cap = actual_caps[i]['captions'][j][0]
                break
        if act_cap != '':
            break
    dict_act[actual_caps[i]['video_id']] = [act_cap]

Meteor_score, Meteor_scores = Meteor().compute_score(dict_act, dict_res)
print(f'Meteor: {Meteor_score*100}')
dict_metrics_res = {}
dict_metrics_res['Meteor'] = {}
dict_metrics_res['Meteor']['Meteor_score'] = Meteor_score
dict_metrics_res['Meteor']['Meteor_scores'] = Meteor_scores

Bleu_score, Bleu_scores = Bleu(4).compute_score(dict_act, dict_res)
for i in range(0, len(Bleu_score)):
    print(f'Bleu_{i+1}: {Bleu_score[i]*100}')
    dict_metrics_res['Bleu_' + str(i+1)] = {}
    dict_metrics_res['Bleu_' + str(i+1)]['Bleu_' + str(i+1) + '_score'] = Bleu_score[i]
    dict_metrics_res['Bleu_' + str(i+1)]['Bleu_' + str(i+1) + '_scores'] = Bleu_scores[i]

Rouge_score, Rouge_scores = Rouge().compute_score(dict_act, dict_res)
print(f'Rouge: {Rouge_score*100}')
dict_metrics_res['Rouge'] = {}
dict_metrics_res['Rouge']['Rouge_score'] = Rouge_score
dict_metrics_res['Rouge']['Rouge_scores'] = Rouge_scores.tolist()

Cider_score, Cider_scores = Cider().compute_score(dict_act, dict_res)
print(f'Cider: {Cider_score*100}')
dict_metrics_res['Cider'] = {}
dict_metrics_res['Cider']['Cider_score'] = Cider_score
dict_metrics_res['Cider']['Cider_scores'] = Cider_scores.tolist()

eval_json = json.dumps(dict_metrics_res, indent = 4)
with open(os.path.join(result_path, 'MSVD16_1500_eval_metric_results.json'), 'w') as outfile:
    outfile.write(eval_json)

Meteor: 12.120169174763706
{'testlen': 3111, 'reflen': 2823, 'guess': [3111, 2441, 1771, 1101], 'correct': [965, 281, 102, 10]}
ratio: 1.1020191285862195
Bleu_1: 31.018964963024427
Bleu_2: 18.896567421835964
Bleu_3: 12.716935964685753
Bleu_4: 6.574156427129189
Rouge: 31.784602716189603
Cider: 38.80229023907884
